In [ ]:
# upload fasta
import os
import shutil
from google.colab import files

# upload dir
os.makedirs('/content/input_fasta/', exist_ok=True)

# upload
uploaded = files.upload()

# save fasta to dir
for filename in uploaded.keys():
    file_path = os.path.join('/content/input_fasta/', filename)
    with open(file_path, 'wb') as f:
        f.write(uploaded[filename])



In [ ]:
# Git clone original files
!git clone https://github.com/orca233/DeepSS2GO_v2_colab.git


# # mv fasta to dir
for filename in uploaded.keys():
    source = os.path.join('/content/input_fasta/', filename)
    destination = '/content/DeepSS2GO_v2_colab/pub_data/data_new/new_aa.fa'
    shutil.move(source, destination)
    print(f"File copied to {destination}")


In [ ]:
!python -V
!pip -V

In [ ]:
# Since colab use python 3.10 by default, we need to reinstall py3.8

# Since py3.8 does not have '_ctypes', so need to pre-install 'libffi-dev'
!sudo apt-get install libffi-dev

# 'pyenv' installation & configuration
!apt-get install -y build-essential checkinstall
!apt-get install -y libreadline-gplv2-dev libncursesw5-dev libssl-dev libsqlite3-dev tk-dev libgdbm-dev libc6-dev libbz2-dev libffi-dev zlib1g-dev
!curl https://pyenv.run | bash

# Configuring environment variables：
import os

os.environ["PATH"] = "/root/.pyenv/bin:/root/.pyenv/shims:/root/.pyenv/plugins/pyenv-virtualenv/shims:" + os.environ["PATH"]
!pyenv init -
!pyenv virtualenv-init -

# install Python 3.8
!pyenv install 3.8.12
!pyenv global 3.8.12

# reinstall pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python get-pip.py


In [ ]:
# environment
# py lib
!pip install click==8.1.3
!pip install fair-esm==2.0.0
!pip install matplotlib==3.7.1
!pip install numpy==1.24.3
!pip install pandas==1.3.5
!pip install scikit-learn==1.2.2
!pip install scipy==1.10.1
!pip install seaborn==0.12.2
!pip install torch==1.8.0+cu111 torchaudio==0.8.0 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchsummary==1.5.1
!pip install tqdm==4.65.0
!pip install transformers==4.29.2

# BLAST
!apt-get install ncbi-blast+

# Diamond
!wget http://github.com/bbuchfink/diamond/releases/download/v2.1.7/diamond-linux64.tar.gz
!tar xzf diamond-linux64.tar.gz
!cp diamond /usr/local/bin/
!diamond version  # confirm success

In [ ]:
!python -V
!pip -V

In [ ]:
#################################
### PART1: from aa to ss8 #######
#################################

In [ ]:
%cd /content/DeepSS2GO_v2_colab/PredictNew/s1_DataPreprocessing_PredictNew
!python step1_fa2pkl.py

In [ ]:
!python step2_New_x_SPOT1DLM.py

In [ ]:
!python step3_SPOT1DLM_generate_esm.py

In [ ]:
# For running step4 - prot trans
!pip install transformers sentencepiece

from transformers import T5Tokenizer, T5EncoderModel
# Force re-download model
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False, force_download=True)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50", force_download=True)

In [ ]:
!python step4_SPOT1DLM_generate_prottrans.py

In [ ]:
# For running step5 - download SPOT-1D-LM checkpoint

# download tools
!pip install requests
!apt-get install xz-utils

import requests

url = "https://huggingface.co/orca233/DeepSS2GO/resolve/main/SPOT-LM-checkpoints.xz"

# download path
file_path = "/content/DeepSS2GO_v2_colab/pub_data/SPOT1DLM_checkpoints/SPOT-LM-checkpoints.xz"

!mkdir -p /content/DeepSS2GO_v2_colab/pub_data/SPOT1DLM_checkpoints

# download
response = requests.get(url, stream=True)
with open(file_path, "wb") as file:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            file.write(chunk)

# decompression
import os

os.system(f"tar --strip-components=1 -xvJf {file_path} -C /content/DeepSS2GO_v2_colab/pub_data/SPOT1DLM_checkpoints")


In [ ]:
!python step5_SPOT1DLM_run_inference.py

In [ ]:
!python step6_SPOT1DLM_csv_2_aass3ss8.py

In [ ]:
!python step7_aa_2_ss3ss8.py

In [ ]:
!python step8_pkl2fa.py

In [ ]:
###############################
####### PART2: predict ########
###############################

In [ ]:
# Download trained model
!wget -O /content/s3_AlphaBeta_bpccmf.tar.gz https://huggingface.co/orca233/DeepSS2GO/resolve/main/s3_AlphaBeta_bpccmf.tar.gz

# decompression
!tar -xzvf /content/s3_AlphaBeta_bpccmf.tar.gz -C /content/DeepSS2GO_v2_colab/PredictNew/s3_PredictNew_AlphaBeta/s3_AlphaBeta_bpccmf --strip-components=1


In [ ]:
%cd /content/DeepSS2GO_v2_colab/PredictNew/s3_PredictNew_AlphaBeta/s3_AlphaBeta_bpccmf/

In [ ]:
!bash step6_cpData_Diamond4New.sh

In [ ]:
!bash step7_PredictAlphaBeta_New.sh

In [ ]:
# download results_bp/cc/mf.csv
from google.colab import files

files.download('/content/DeepSS2GO_v2_colab/PredictNew/s3_PredictNew_AlphaBeta/s3_AlphaBeta_bpccmf/data/results_bp.csv')

files.download('/content/DeepSS2GO_v2_colab/PredictNew/s3_PredictNew_AlphaBeta/s3_AlphaBeta_bpccmf/data/results_cc.csv')

files.download('/content/DeepSS2GO_v2_colab/PredictNew/s3_PredictNew_AlphaBeta/s3_AlphaBeta_bpccmf/data/results_mf.csv')
